In [105]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep
from random import randint

In [106]:
# Define a dictionary headers to store the User-Agent string for the request
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

In [130]:
# Read the player links from the csv file
df_links = pd.read_csv('output/player_links.csv')

# Convert the 'URL' column of the dataframe to a list
links = df_links['URL'].tolist()

In [131]:
# List to store data of each player in the form of a dictionary
list_of_row_dicts = []

# Loop through each link to scrape player data
for link in links:

    # Keep trying until the request is successful
    while True:
        try:
            # Make a GET request to the link
            request = rq.get(link,headers=headers)
        
            # Use BeautifulSoup to parse the HTML content of the page
            soup = bs(request.text, 'html.parser')

            # Find all <span> tags in the HTML
            title_spans = soup.find_all('span')
                       
            # Break out of the loop if the request is successful
            break
        except AttributeError:
            # Print error message and wait for 10 seconds before retrying
            print('Index Error : Sleeping for 10 seconds before retrying')
            sleep(10)
            
    try:
        name = ' '.join([word for word in soup.find('h1').text.split() if not any(i.isdigit() for i in word)]).strip()
    except IndexError:
        name = None
        print ('Name not found')

    try:
        date_of_birth = [span.find_next('span').text for span in title_spans if 'Date of birth/Age:' in span.text][0].strip()
    except IndexError:
        date_of_birth = None
        print ('DoB not found')

    try:
        citizenship = [span.find_next('span').text.strip().split('\xa0\xa0') for span in title_spans if 'Citizenship:' in span.text][0]
    except AttributeError:
        citizenship = None
        print ('Citizenship not found')

    try:
        current_club = [span.find_next('span').text for span in title_spans if 'Current club:' in span.text][0].strip()
    except IndexError:
        current_club = None
        print ('Current club not found')

    try:
        outfitter = [span.find_next('span').text for span in title_spans if 'Outfitter:' in span.text][0].strip()
    except IndexError:
        outfitter = None
        print ('Outfitter not found')

    try:
        mv = [span.next_sibling.strip() for span in title_spans if '€' in span.text][0].strip()
        korm = [span.find_next('span').text for span in title_spans if '€' in span.text][0].strip()
        mv = mv + korm
    except IndexError:
        mv = None
        print ('MV not found')
            
    row_dic = {
    'PLAYER_URL' : link,
    'NAME' : name,
    'DATE_OF_BIRTH' : date_of_birth,
    'CITIZENSHIP': citizenship,
    'CURRENT_CLUB' : current_club,
    'OUTFITTER' : outfitter,
    'MV' : mv
    }
    list_of_row_dicts.append(row_dic)
            
    print(name,',',date_of_birth,',',citizenship,',',current_club,',',outfitter,',',mv)
    sleep(randint(1,3))

df = pd.DataFrame(list_of_row_dicts)

Lucas Perri , Dec 10, 1997 (27) , ['Brazil', 'Italy'] , Olympique Lyon , adidas , 10.00m
Outfitter not found
Rémy Descamps , Jun 25, 1996 (28) , ['France'] , Olympique Lyon , None , 1.50m
Outfitter not found
Lassine Diarra , Nov 11, 2002 (22) , ['Mali'] , Olympique Lyon B , None , 25k
Outfitter not found
Moussa Niakhaté , Mar 8, 1996 (29) , ['Senegal', 'France'] , Olympique Lyon , None , 15.00m
Outfitter not found
Duje Caleta-Car , Sep 17, 1996 (28) , ['Croatia'] , Olympique Lyon , None , 8.00m
Warmed Omari , Apr 23, 2000 (24) , ['Comoros', 'France'] , Olympique Lyon , Puma , 4.00m
Outfitter not found
Abner , May 27, 2000 (24) , ['Brazil'] , Olympique Lyon , None , 9.00m
Nicolás Tagliafico , Aug 31, 1992 (32) , ['Argentina', 'Italy'] , Olympique Lyon , Nike , 7.00m
Ainsley Maitland-Niles , Aug 29, 1997 (27) , ['England', 'Jamaica'] , Olympique Lyon , Nike , 10.00m
Outfitter not found
Saël Kumbedi , Mar 26, 2005 (20) , ['France', 'DR Congo'] , Olympique Lyon , None , 6.00m
Outfitter not

In [132]:
df.to_csv('output/player_info.csv')